# Import Libaries

In [15]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pickle

In [16]:
data = pd.read_feather('output.feather')
data

,abaqus,ablation,ableton live,abnormal psychology,absorption,ac motors,academic advising,academic english,academic integrity,academic writing,...,z axis,zbrush,zemax,zendesk,zeta potential,zoning,zoology,zoom,zumba,zumba fitness
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2481,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
resume = pd.read_csv('Resume.csv')
y = resume['Category']
data['y'] = y
data

,abaqus,ablation,ableton live,abnormal psychology,absorption,ac motors,academic advising,academic english,academic integrity,academic writing,...,zbrush,zemax,zendesk,zeta potential,zoning,zoology,zoom,zumba,zumba fitness,y
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AVIATION
2480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AVIATION
2481,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AVIATION
2482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AVIATION


# Aggregation by Class

In [18]:
agg_data = data.groupby(['y']).sum()
agg_data

,abaqus,ablation,ableton live,abnormal psychology,absorption,ac motors,academic advising,academic english,academic integrity,academic writing,...,z axis,zbrush,zemax,zendesk,zeta potential,zoning,zoology,zoom,zumba,zumba fitness
y,,,,,,,,,,,,,,,,,,,,,
ACCOUNTANT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ADVOCATE,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
AGRICULTURE,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
APPAREL,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
ARTS,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
AUTOMOBILE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AVIATION,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BANKING,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
BPO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
agg_data = agg_data.T
agg_data

y,ACCOUNTANT,ADVOCATE,AGRICULTURE,APPAREL,ARTS,AUTOMOBILE,AVIATION,BANKING,BPO,BUSINESS-DEVELOPMENT,...,DIGITAL-MEDIA,ENGINEERING,FINANCE,FITNESS,HEALTHCARE,HR,INFORMATION-TECHNOLOGY,PUBLIC-RELATIONS,SALES,TEACHER
abaqus,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
ablation,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
ableton live,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
abnormal psychology,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
absorption,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zoning,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zoology,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zoom,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
zumba,0,0,0,0,1,0,0,0,0,0,...,0,0,0,6,0,0,0,0,0,0


In [20]:
agg_data.describe()

y,ACCOUNTANT,ADVOCATE,AGRICULTURE,APPAREL,ARTS,AUTOMOBILE,AVIATION,BANKING,BPO,BUSINESS-DEVELOPMENT,...,DIGITAL-MEDIA,ENGINEERING,FINANCE,FITNESS,HEALTHCARE,HR,INFORMATION-TECHNOLOGY,PUBLIC-RELATIONS,SALES,TEACHER
count,5377.000000,5377.000000,5377.000000,5377.000000,5377.000000,5377.000000,5377.000000,5377.000000,5377.000000,5377.000000,...,5377.000000,5377.000000,5377.000000,5377.000000,5377.000000,5377.000000,5377.000000,5377.000000,5377.000000,5377.000000
mean,0.516645,0.466989,0.299795,0.361540,0.387019,0.144876,0.498605,0.527618,0.116236,0.538962,...,0.468663,0.678259,0.545285,0.407104,0.545471,0.448763,0.768272,0.495816,0.342756,0.295518
std,3.650994,2.673113,1.480323,2.245828,2.124896,0.834316,2.849990,3.126254,0.651856,3.626439,...,2.532856,2.922848,3.669149,2.546146,2.867457,3.047068,3.787542,3.175138,2.528605,2.030617
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,102.000000,100.000000,54.000000,84.000000,79.000000,30.000000,97.000000,106.000000,22.000000,119.000000,...,79.000000,97.000000,118.000000,96.000000,107.000000,104.000000,113.000000,99.000000,94.000000,76.000000


# Identify Significant Skills

We used percentile method here, any value smaller than 90 percentile is not significant. 

In [21]:
# We want to find among all the skills that appear at least once, what is the 95 percentile value for a particular job
print("Accountant:", np.percentile(agg_data['ACCOUNTANT'][agg_data['ACCOUNTANT'] != 0], 95))
print("BPO:", np.percentile(agg_data['BPO'][agg_data['BPO'] != 0], 95))

Accountant: 17.0
BPO: 4.899999999999977


In [22]:
# Accountant significant skills
temp = agg_data['ACCOUNTANT'][agg_data['ACCOUNTANT'] > np.percentile(agg_data['ACCOUNTANT'][agg_data['ACCOUNTANT'] != 0], 95)]
temp

auditing                    33
balance sheet               47
banking                     29
billing                     51
bookkeeping                 23
budgeting                   28
business administration     50
cash management             18
closing                     59
customer service            32
depreciation                24
filing                      35
finance                     63
financial analysis          24
financial management        23
forecasting                 25
general ledger              81
inquiry                     26
invoicing                   19
leadership                  24
m                           25
management                 102
marketing                   18
microsoft excel             18
microsoft office            49
planning                    38
r                           22
reconciliation              99
research                    63
source                      19
time management             26
track                       48
variance

In [23]:
print(temp.max(), temp.min())

102 18


In [24]:
# Scaling
temp.apply(lambda x: (x - temp.min()) / (temp.max() - temp.min()))

auditing                   0.178571
balance sheet              0.345238
banking                    0.130952
billing                    0.392857
bookkeeping                0.059524
budgeting                  0.119048
business administration    0.380952
cash management            0.000000
closing                    0.488095
customer service           0.166667
depreciation               0.071429
filing                     0.202381
finance                    0.535714
financial analysis         0.071429
financial management       0.059524
forecasting                0.083333
general ledger             0.750000
inquiry                    0.095238
invoicing                  0.011905
leadership                 0.071429
m                          0.083333
management                 1.000000
marketing                  0.000000
microsoft excel            0.000000
microsoft office           0.369048
planning                   0.238095
r                          0.047619
reconciliation             0

In [25]:
# If value > 0.7, skill level required is 5; 
# If 0.7 >= value > 0.3, skill level required is 4;
# If 0.3 >= value, skill level required is 3;
temp2 = temp.apply(lambda x: (x - temp.min()) / (temp.max() - temp.min())).apply(lambda x: 5 if x > 0.7 else 4 if x > 0.3 else 3)
temp2

auditing                   3
balance sheet              4
banking                    3
billing                    4
bookkeeping                3
budgeting                  3
business administration    4
cash management            3
closing                    4
customer service           3
depreciation               3
filing                     3
finance                    4
financial analysis         3
financial management       3
forecasting                3
general ledger             5
inquiry                    3
invoicing                  3
leadership                 3
m                          3
management                 5
marketing                  3
microsoft excel            3
microsoft office           4
planning                   3
r                          3
reconciliation             5
research                   4
source                     3
time management            3
track                      4
variance analysis          3
Name: ACCOUNTANT, dtype: int64

In [26]:
# show the advanced skills to have so that you are better than most of your peers
temp2[temp2 == 5]

general ledger    5
management        5
reconciliation    5
Name: ACCOUNTANT, dtype: int64

In [27]:
skills_required = {}
for col in agg_data.columns:
    print(col)
    series = agg_data[col][agg_data[col] > np.percentile(agg_data[col][agg_data[col] != 0], 95)]
    scaled_series = series.apply(lambda x: (x - series.min()) / (series.max() - series.min()))
    binned_series = scaled_series.apply(lambda x: 5 if x > 0.7 else 4 if x > 0.3 else 3)
    skills_required[col] = binned_series.to_dict()

ACCOUNTANT
ADVOCATE
AGRICULTURE
APPAREL
ARTS
AUTOMOBILE
AVIATION
BANKING
BPO
BUSINESS-DEVELOPMENT
CHEF
CONSTRUCTION
CONSULTANT
DESIGNER
DIGITAL-MEDIA
ENGINEERING
FINANCE
FITNESS
HEALTHCARE
HR
INFORMATION-TECHNOLOGY
PUBLIC-RELATIONS
SALES
TEACHER


In [28]:
skills_required

{'ACCOUNTANT': {'auditing': 3,
  'balance sheet': 4,
  'banking': 3,
  'billing': 4,
  'bookkeeping': 3,
  'budgeting': 3,
  'business administration': 4,
  'cash management': 3,
  'closing': 4,
  'customer service': 3,
  'depreciation': 3,
  'filing': 3,
  'finance': 4,
  'financial analysis': 3,
  'financial management': 3,
  'forecasting': 3,
  'general ledger': 5,
  'inquiry': 3,
  'invoicing': 3,
  'leadership': 3,
  'm': 3,
  'management': 5,
  'marketing': 3,
  'microsoft excel': 3,
  'microsoft office': 4,
  'planning': 3,
  'r': 3,
  'reconciliation': 5,
  'research': 4,
  'source': 3,
  'time management': 3,
  'track': 4,
  'variance analysis': 3},
 'ADVOCATE': {'active listening': 3,
  'advocacy': 3,
  'bilingual': 3,
  'billing': 3,
  'business administration': 3,
  'c': 3,
  'case management': 3,
  'cash register': 3,
  'closing': 3,
  'coaching': 3,
  'collaboration': 3,
  'conflict resolution': 3,
  'crisis intervention': 3,
  'critical thinking': 3,
  'customer service'

In [29]:
skills_file = open('skills_required.pkl', 'wb')
pickle.dump(skills_required, skills_file)
skills_file.close()

In [2]:
with open('skills_required', 'rb') as pickle_file:
    content = pickle.load(pickle_file)